In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "0,1"  # Set the GPU 1 to use

from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
#여기까지 pytorch 기본 아래로는 따로 추가
from torch.utils.data import Dataset, DataLoader
from konlpy.tag import Okt
from tokenizers.implementations import SentencePieceBPETokenizer
import import_ipynb
import SetData
from mtDataset import MTDataset
#import model_class as mc
#from SentencepieceTokenizer import sentencepiece_tokenizer

importing Jupyter notebook from SetData.ipynb
importing Jupyter notebook from mtDataset.ipynb


/usr/anaconda3/envs/torch109_py38/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def save_model(save_path, model_name, model, optimizer, valid_loss):
    # 모델 state_dict 저장
    torch.save(model.state_dict(), save_path + model_name + '_state_dict.pt')  

In [5]:
#en 토크나이저
corpus = '/etc/jupyterhub/pythonex/Paper_2023/Data/source.txt'

sentencepiece_tokenizer = SentencePieceBPETokenizer(add_prefix_space = True)
sentencepiece_tokenizer.train(
    files = corpus,
    vocab_size = 20004,
    min_frequency = 2,
    special_tokens = ['<unk>', '<pad>', '<bos>', '<eos>']
)
vocab = sentencepiece_tokenizer.get_vocab()
print(len(vocab))
#print(sorted(vocab, key=lambda x: vocab[x]))

#sentencepiece_tokenizer.save_model('vocab', 'source_bpe_new')

#토크나이저를 통해 문장을 토큰으로 변환하는 함수
def morphs(sentence):
    return sentencepiece_tokenizer.encode(sentence).tokens

#kr 토크나이저
corpus = '/etc/jupyterhub/pythonex/Paper_2023/Data/target.txt'

sentencepiece_tokenizer_target = SentencePieceBPETokenizer(add_prefix_space = True)
sentencepiece_tokenizer_target.train(
    files = corpus,
    vocab_size = 20004,
    min_frequency = 2,
    special_tokens = ['<unk>', '<pad>', '<bos>', '<eos>']
)
vocab = sentencepiece_tokenizer_target.get_vocab()
print(len(vocab))

#토크나이저를 통해 문장을 토큰으로 변환하는 함수
def morphs_target(sentence):
    return sentencepiece_tokenizer_target.encode(sentence).tokens




20004
20004





In [4]:
morphs('제3조, 제7조부터 제9조까지 및 제9조의2에 따른 등록신청, 재외국민등록부 등본 교부신청, 변경신고, 이동신고 및 귀국신고는 대통령령으로 정하는 바에 따라 다음 각 호의 방법으로 할 수 있다.')

['▁제3',
 '조,',
 '▁제7',
 '조부터',
 '▁제9조',
 '까지',
 '▁및',
 '▁제9',
 '조의2에',
 '▁따른',
 '▁등록신청',
 ',',
 '▁재외국민',
 '등록',
 '부',
 '▁등본',
 '▁교부',
 '신청',
 ',',
 '▁변경',
 '신고,',
 '▁이동',
 '신고',
 '▁및',
 '▁귀국',
 '신고',
 '는',
 '▁대통령령으로',
 '▁정하는',
 '▁바에',
 '▁따라',
 '▁다음',
 '▁각',
 '▁호의',
 '▁방법으로',
 '▁할',
 '▁수',
 '▁있다.']

In [3]:
gpu_count = torch.cuda.device_count()
DEVICE =  [torch.device("cuda:0" if torch.cuda.is_available() else "cpu"), torch.device("cuda:1" if torch.cuda.is_available() else "cpu")]
# 스페셜 토큰 인덱스 지정
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3

# 데이터의 파일 정보
train_file_path = "/etc/jupyterhub/pythonex/Paper_2023/Data/train.txt"
valid_file_path = "/etc/jupyterhub/pythonex/Paper_2023/Data/valid.txt"

# 데이터 불러오기
data_pairs = SetData.load_file(train_file_path)
train_dataset = MTDataset(data_pairs)
data_pairs = SetData.load_file(valid_file_path)
valid_dataset = MTDataset(data_pairs)


In [6]:
#데이터 구하고 처리
from torchtext.vocab import build_vocab_from_iterator
from typing import Iterable, List

SRC_LANGUAGE = 'ko'
TGT_LANGUAGE = 'en'

token_transform = {}
vocab_transform = {}

#토크나이저 생성 후 정의한 토큰화 함수를 딕셔너리에 저장합니다.
token_transform[SRC_LANGUAGE] = morphs
token_transform[TGT_LANGUAGE] = morphs_target 

val_iter = SetData.make_iter(valid_file_path)

# 토큰 목록을 생성하기 위한 헬퍼(helper) 함수
def yield_tokens(data_iter: Iterable, language: str) -> List[str]:
    language_index = {SRC_LANGUAGE: 0, TGT_LANGUAGE: 1}
    
    for data_sample in data_iter:
        yield token_transform[language](data_sample[language_index[language]])

# 토큰들이 어휘집(vocab)에 인덱스 순서대로 잘 삽입되어 있는지 확인
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    # 학습용 데이터 반복자(iterator)
    train_iter = SetData.make_iter(train_file_path)
    # torchtext의 Vocab(어휘집) 객체 생성
    vocab_transform[ln] = build_vocab_from_iterator(yield_tokens(train_iter, ln),
                                                    min_freq=1,
                                                    specials=special_symbols,
                                                    special_first=True)

# UNK_IDX를 기본 인덱스로 설정. 이 인덱스는 토큰을 찾지 못하는 경우에 반환
# 만약 기본 인덱스를 설정하지 않으면 어휘집(Vocabulary)에서 토큰을 찾지 못하는 경우
# RuntimeError가 발생
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    vocab_transform[ln].set_default_index(UNK_IDX)

In [7]:
print(len(vocab_transform['ko']))
print(len(vocab_transform['en']))

19992
19917


In [9]:
cnt = 0
for data_sample in train_dataset:
  print(data_sample)
  print(type(data_sample))
  cnt += 1
  if (cnt > 5):
    break
    
for data_sample in valid_dataset:
  print(data_sample)
  print(type(data_sample))
  cnt += 1
  if (cnt > 5):
    break
    
print(len(train_dataset))
print(len(valid_dataset))

('② 「건축법」 제4조제5항, 제5조제1항, 제6조, 제8조, 제12조제1항, 제13조제2항, 제14조제1항 각 호 외의 부분, 같은 항 제4호·제5호, 제16조제1항 본문·단서, 같은 조 제2항, 제17조제2항, 제19조제3항 본문·단서, 같은 조 제4항 각 호 외의 부분, 같은 조 제6항, 제20조제2항제3호, 같은 조 제3항·제5항, 제22조제2항 각 호 외의 부분 본문, 같은 조 제3항제2호, 제23조제1항제3호, 같은 조 제4항, 제24조제5항, 제25조제1항 전단, 같은 조 제4항부터 제6항까지, 같은 조 제8항, 제26조, 제27조제1항부터 제3항까지, 제28조제1항, 제29조제1항, 제34조, 제37조제1항·제2항, 제40조제4항, 제41조제1항, 제42조제1항 단서, 제43조제1항 각 호 외의 부분, 같은 조 제2항, 제44조제1항제2호, 같은 조 제2항, 제45조제1항 각 호 외의 부분 본문, 같은 조 제3항, 제46조제1항 단서, 같은 조 제2항, 제54조제1항 본문, 제57조제1항, 제58조, 제59조제1항제1호·제2호, 같은 조 제2항, 제60조제1항 본문·단서, 제61조제1항, 같은 조 제2항 각 호 외의 부분, 같은 조 제3항 각 호 외의 부분, 같은 항 제8호, 제62조, 제64조제1항 전단·후단, 같은 조 제2항 본문·단서, 제69조제1항제1호나목, 같은 항 제2호나목·다목, 제70조제2호·제3호, 제71조제1항제4호 후단, 같은 항 제6호 후단, 같은 항 제7호, 같은 조 제5항, 같은 조 제7항 후단, 제72조제1항 각 호 외의 부분 후단, 같은 조 제5항 전단, 같은 조 제7항, 같은 조 제8항 후단, 제73조제1항제2호, 같은 조 제3항, 제75조제2항 전단·후단, 제78조제4항, 제79조제2항 단서, 같은 조 제4항, 제80조제1항 각 호 외의 부분 단서, 같은 항 제2호, 제83조제1항·제3항, 제85조제1항제5호, 제88조제1항제7호 및 제102조제2항에서 대통령령 또는 국토교통부령으로 정하도록 한 사항은 도조례로 

In [10]:
#학습하는 동안, 모델이 예측하는 동안 정답(이후 출현하는 단어)을 보지 못하도록 하는 후속 단어 마스크(subsequent word mask)
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE[0])) == 1).transpose(0, 1) #상삼각행렬을 이용해 1로 채워진 텐서의 하위 값을 0으로 변경하고, 행/열 위치를 변경
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

#출발어와 도착어의 패딩(padding) 토큰들 또한 숨겨야 합니다. 아래에 두 가지 모두를 처리
def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE[0]).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [11]:
# 단어 순서 개념(notion)을 토큰 임베딩에 도입하기 위한 위치 인코딩(positional encoding)을 위한 헬퍼 모듈(Module)
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2) # 뒤에서 2번째 자리에 1로 채워진 차원 추가

        self.dropout = nn.Dropout(dropout)
        #self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# 입력 인덱스의 텐서를 해당하는 토큰 임베딩의 텐서로 변환하기 위한 헬퍼 모듈(Module)
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq 신경망
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

In [12]:
#데이터 반복자(iterator)는 원시 문자열의 쌍을 생성
#이 문자열 쌍들을 이전에 정의한 Seq2Seq 신경망에서 처리할 수 있도록 텐서 묶음(batched tensor)으로 변환
#이제 원시 문자열들의 묶음(batch)을 텐서 묶음으로 변환하여 모델에 직접 전달할 수 있도록 하는 대응어(collate) 함수를 정의
from torch.nn.utils.rnn import pad_sequence

# 순차적인 작업들을 하나로 묶는 헬퍼 함수
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

# BOS/EOS를 추가하고 입력 순서(sequence) 인덱스에 대한 텐서를 생성하는 함수
def tensor_transform(token_ids: List[int]):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))

# 출발어(src)와 도착어(tgt) 문자열들을 텐서 인덱스로 변환하는 변형(transform)
text_transform = {}
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[ln] = sequential_transforms(token_transform[ln], # 토큰화 함수(Tokenization)
                                               vocab_transform[ln], # 수치화 함수(Numericalization)
                                               tensor_transform) # BOS/EOS를 추가하고 텐서를 생성


# 데이터를 텐서로 조합(collate)하는 함수
def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip("\n")))
        tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n")))

    #batch로 묶이는 데이터들의 길이가 다르기 때문에 패딩처리
    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch


In [13]:
#모델의 매개변수를 정의하고 객체를 생성
torch.manual_seed(0)

SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
#BATCH_SIZE = 128
BATCH_SIZE = 64
NUM_ENCODER_LAYERS = 6
NUM_DECODER_LAYERS = 6

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE[0])

#학습 단계에서 사용할 손실 함수(loss function)를 교차 엔트로피 손실(cross-entropy loss)로 정의하고, 옵티마이저(optimizer)도 정의
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn)
val_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn)

In [15]:
#각 에폭(epoch)마다 호출할 학습 및 검증(evaluation) 단계를 정의
def train_epoch(model, optimizer, train_dataloader, global_step, best_valid_loss):
    model.train()
    losses = 0
    
    for src, tgt in train_dataloader:
        start_time = timer()
        src = src.to(DEVICE[0])
        tgt = tgt.to(DEVICE[0])

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)
        
        # gradient를 0으로 초기화
        optimizer.zero_grad()

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        # W와 b를 업데이트
        optimizer.step()
        losses += loss.item()
        
        end_time = timer()
        
        global_step+= 1
        #5000 step에 한번씩 로그 저장 및 로스 값을 비교하여 모델 저장
        if(global_step % 5000 == 0 and not global_step == 0):
            train_loss = losses / len(train_dataloader)
            val_loss = evaluate(model, val_dataloader)            
            #print(f"Step: {global_step}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, Batch time = {(end_time - start_time):.3f}s")
            SetData.write_file('train_status.txt', f"Step: {global_step}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, Batch time = {(end_time - start_time):.3f}s")
                        
            if best_valid_loss > val_loss:
                best_valid_loss = val_loss
                print('save model')
                save_model('models', '/mt-model-old-v20k_v20k', model, optimizer, best_valid_loss)

    return losses / len(train_dataloader), global_step, best_valid_loss


def evaluate(model, val_dataloader):
    model.eval()
    losses = 0
    
    with torch.no_grad():
        for src, tgt in val_dataloader:
            src = src.to(DEVICE[0])
            tgt = tgt.to(DEVICE[0])

            tgt_input = tgt[:-1, :]

            src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

            logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

            tgt_out = tgt[1:, :]
            loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
            losses += loss.item()

    return losses / len(val_dataloader)

In [ ]:
#학습
from timeit import default_timer as timer
NUM_EPOCHS = 25

best_valid_loss = float("Inf")
global_step = 0
for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss, global_step, best_valid_loss = train_epoch(transformer, optimizer, train_dataloader, global_step, best_valid_loss)
    val_loss = evaluate(transformer, val_dataloader)
    end_time = timer()
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))
    SetData.write_file('train_status.txt', f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, Epoch time = {(end_time - start_time):.3f}s")
    #if best_valid_loss > val_loss:
        #best_valid_loss = val_loss
        #print('save model')
        #save_model('models', '/mt-model-full-v30k_v30k', transformer, optimizer, best_valid_loss)


save model
save model
save model
Epoch: 1, Train loss: 2.928, Val loss: 3.189, Epoch time = 1922.667s
